In [ ]:
import requests
import json
from pathlib import Path

GMAPS_API_KEY = (Path.home() / "api_keys" / "google_maps.txt").read_text().strip()

# Sampling a street view dataset has three stages:
# Stage 1: raw
#          Sample raw points (not necessarily SV panoramas)
# Stage 2: sv_cleaned
#          Transform raw points into closest SV panoramas. Filter out non-existent points
# Stage 3: parameterized
#          Select heading, pitch, etc. for each panorama.
# Stage 4: final
#          Remove points that are not in the S2 cell mapping (no point in downloading them).

# Transform points to nearest Street View panorama ID
def streetview_stage2_clean_annotate(raw_df):
    sv_lat = []
    sv_lng = []
    sv_pano = []
    sv_status = []
    for row in raw_df.itertuples():
        # Call the Street View API to get the nearest panorama
        result = requests.get(
            "https://maps.googleapis.com/maps/api/streetview/metadata"
            f"?location={row['lat']},{row['lng']}"
            f"&radius=50"
            f"&key={GMAPS_API_KEY}"
        ).json()
        sv_status.append(result["status"])
        sv_lat.append(result.get("location", {}).get("lat"))
        sv_lng.append(result.get("location", {}).get("lng"))
        sv_pano.append(result.get("pano_id"))

    out_df = raw_df.rename(
        columns={"lat": "raw_lat", "lng": "raw_lng"},
        inplace=False,
    )
    out_df['lat'] = sv_lat
    out_df['lng'] = sv_lng
    out_df['pano_id'] = sv_pano
    out_df['status'] = sv_status
    #return out_df[out_df['status'] == "OK"]
    out_df = out_df.query("status == 'OK'")

    # Randomize in case the input isn't
    return out_df.sample(frac=1)

# Parameterize a SV dataframe
def streetview_stage3_parameterize(sv_df):
    sv_df['fov'] = 45
    sv_df['pitch'] = 0

    # Randomize the heading
    sv_df['heading'] = np.random.randint(0, 360, len(sv_df))

    # Generate a path to save each image to
    paths = []
    for row in sv_df.itertuples():
        paths.append(f"sv_{row.Index}.jpg")
    sv_df['_imgpath'] = paths

    return sv_df

# Download panorama images from street view
def streetview_write_point(df_row, output_dir):
    output_dir = Path(output_dir)

    url = (
        "https://maps.googleapis.com/maps/api/streetview"
        f"?pano={df_row['pano_id']}"
        f"&size=640x640"
        f"&fov={df_row['fov']}"
        f"&heading={df_row['heading']}"
        f"&key={GMAPS_API_KEY}"
    )

    # Download image
    jpeg_filename = output_dir / df_row['_imgpath']
    if not os.path.exists(jpeg_filename):
        print("Downloading", jpeg_filename)
        r = requests.get(url, allow_redirects=True)
        with open(jpeg_filename, 'wb') as f:
            f.write(r.content)
    else:
        print("Skipping", jpeg_filename)